In [1]:
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt

π = np.pi

## Basis Definition

Bell_00 / $|\Phi^+\rangle$ = $\frac{1}{\sqrt{2}} \left( |00\rangle + |11\rangle \right)$

Bell_01 / $|\Phi^-\rangle$ = $\frac{1}{\sqrt{2}} \left( |00\rangle - |11\rangle \right)$

Bell_10 / $|\Psi^+\rangle$ =  $\frac{1}{\sqrt{2}} \left( |01\rangle + |10\rangle \right)$

Bell_11 / $|\Psi^-\rangle$ =  $\frac{1}{\sqrt{2}} \left( |01\rangle - |10\rangle \right)$

In [2]:
Bell_00 = qt.bell_state('00')
Bell_01 = qt.bell_state('01')
Bell_10 = qt.bell_state('10')
Bell_11 = qt.bell_state('11')

Magic_1 / $|\Phi_1\rangle$ = $|\Phi^+\rangle$

Magic_2 / $|\Phi_2\rangle$ = $-i|\Phi^-\rangle$

Magic_3 / $|\Psi_3\rangle$ = $|\Psi^-\rangle$

Magic_4 / $|\Psi_4\rangle$ = $-i|\Psi^+\rangle$

In [3]:
#kraus cirac definition
Magic_1 = Bell_00
Magic_2 = -1j * Bell_01
Magic_3 = Bell_11
Magic_4 = -1j * Bell_10

Many different ways to define the magic basis - Here is the one in Vatan Williams 2004, Gates States referenced in the weyl chamber paper (This is different than Kraus Cirac)

Simply stacking the column vectors of the bell basis in matrix representation side by side

In [4]:
#Transform from the standard basis to the magic basis
Q1 = qt.Qobj((1/np.sqrt(2)) * np.array([[1, 1j, 0, 0],
                                       [0, 0, 1j, 1],
                                       [0, 0, 1j, -1],
                                       [1, -1j, 0, 0]], dtype=complex))
Q1

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678+0.j          0.        +0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.70710678j
   0.70710678+0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.70710678j
  -0.70710678+0.j        ]
 [ 0.70710678+0.j          0.        -0.70710678j  0.        +0.j
   0.        +0.j        ]]

In [5]:
#Kraus Cirac
Q2 = qt.Qobj(np.column_stack([Magic_1, Magic_2, Magic_3, Magic_4]))
Q2

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678+0.j          0.        -0.70710678j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.70710678+0.j
   0.        -0.70710678j]
 [ 0.        +0.j          0.        +0.j         -0.70710678+0.j
   0.        -0.70710678j]
 [ 0.70710678+0.j          0.        +0.70710678j  0.        +0.j
   0.        +0.j        ]]

In [6]:
Q2.trans() #this is the change of basis matrix that I computed with the kraus cirac representation of the magic basis
           #this matrix should transform from the standard basis to the magic basis

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.70710678+0.j          0.        +0.j          0.        +0.j
   0.70710678+0.j        ]
 [ 0.        -0.70710678j  0.        +0.j          0.        +0.j
   0.        +0.70710678j]
 [ 0.        +0.j          0.70710678+0.j         -0.70710678+0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        -0.70710678j  0.        -0.70710678j
   0.        +0.j        ]]

## Functions

In [8]:
#http://home.lu.lv/~sd20008/papers/essays/Random%20unitary%20[paper].pdf
def U_matrix(alpha, phi, psi, chi):
    pre = np.exp(1j * alpha)
    a = np.exp(1j * psi) * np.cos(phi) 
    b = np.exp(1j * chi) * np.sin(phi)
    c = -1 * np.exp(-1j * chi) * np.sin(phi)
    d = np.exp(-1j * psi) * np.cos(phi)
    U = pre * np.array([[a, b],
                  [c, d]], 
                 dtype=complex)
    return U

def randSU2():
    alpha = np.random.uniform(0, 2*π)
    psi = np.random.uniform(0, 2*π)
    chi = np.random.uniform(0, 2*π)
    phi = np.random.uniform(0, π/2)
    x = qt.Qobj(U_matrix(alpha, phi, psi, chi))
    return x

#input a np.array state in the standard basis
#computed change of basis matrix on my ipad, turns out it's the transpose of Q... not sure why atm
def concurrence(state):
    basistransform = qt.Qobj(Q2.trans()*state)
    conc = np.abs(basistransform[0][0][0]**2 + 
           basistransform[1][0][0]**2 + 
           basistransform[2][0][0]**2 + 
           basistransform[3][0][0]**2)
    return conc

def can_decomp(tx, ty, tz):
    xx = qt.Qobj(np.kron(qt.sigmax(), qt.sigmax()))
    yy = qt.Qobj(np.kron(qt.sigmay(), qt.sigmay()))
    zz = qt.Qobj(np.kron(qt.sigmaz(), qt.sigmaz()))

    return ((tx*xx + ty*yy + tz*zz)*(-1j*π/2)).expm()

In [9]:
#check that randomly generated matrix in SU(2) is unitary
temp = []
for i in range(0, 100, 1):
    x = randSU2()
    temp.append(x*x.dag() == qt.Qobj(np.identity(2)))
    
    if (x*x.dag() != qt.Qobj(np.identity(2))):
        print(1)
        
        
temp = []
for i in range(0, 100, 1):
    x = randSU2()
    y = randSU2()
    xy = qt.Qobj(np.kron(x,y))
    temp.append(xy*xy.dag())
    
    if (xy*xy.dag() != qt.Qobj(np.identity(4))):
        print(1)

## Canonical Gates are diagonal in the magic basis

In [10]:
U = can_decomp(0.83342, 0.234, 0.1234123)
U

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.57749833-0.11337506j  0.        +0.j          0.        +0.j
  -0.15574879-0.79333731j]
 [ 0.        +0.j         -0.10372525-0.02036345j  0.1915644 -0.97577122j
   0.        +0.j        ]
 [ 0.        +0.j          0.1915644 -0.97577122j -0.10372525-0.02036345j
   0.        +0.j        ]
 [-0.15574879-0.79333731j  0.        +0.j          0.        +0.j
   0.57749833-0.11337506j]]

any canonical gate is diagonal in the magic basis

In [14]:
D = Q2.dag() * U * Q2
D

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.42174955-0.90671237j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.73324712+0.67996225j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.29528965+0.95540778j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.08783915-0.99613467j]]

## Nearest Kronecker Decomposition

In [15]:
x1 = randSU2()
y1 = randSU2()

x2 = randSU2()
y2 = randSU2()

In [16]:
#code from Gates and States page 47 for kroneker decomposition

def nearest_kronecker_product(C):
    C = C.reshape(2, 2, 2, 2)
    C = C.transpose(0, 2, 1, 3)
    C = C.reshape(4, 4)
    u, sv, vh = np.linalg.svd(C)
    A = np.sqrt(sv[0]) * u[:, 0].reshape(2, 2)
    B = np.sqrt(sv[0]) * vh[0, :].reshape(2, 2)
    
    return A, B

In [17]:
xy1 = qt.Qobj(np.kron(x1, x1)) #the otimes of two SU(2) matrices span SO(4)
xy2 = qt.Qobj(np.kron(x2, y2))

In [18]:
qt.Qobj(nearest_kronecker_product(np.array(xy1))[0])

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[-0.27769712-0.27594557j -0.06262884+0.9180501j ]
 [ 0.38363698-0.83639765j  0.37579633-0.1097213j ]]

In [19]:
x1

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[ 0.14924306-0.36192286j -0.87259783+0.29208112j]
 [ 0.91948668+0.03581335j  0.24469721+0.30558947j]]

In [20]:
#not quite the same...

In [21]:
xy1*U*xy2

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.13254363-0.67206454j  0.15154078-0.02798945j  0.2161288 +0.4642911j
   0.44137241-0.22344112j]
 [-0.05506688-0.04333929j -0.60435112-0.67584411j -0.04706475-0.10733948j
   0.34912418+0.19354393j]
 [ 0.59943395+0.32233377j -0.16415045-0.18174367j  0.56752565+0.13799809j
  -0.21780898-0.29704236j]
 [ 0.24720487-0.03899261j  0.18603362-0.24425765j -0.49714376-0.36838093j
   0.07674166-0.67405735j]]

## Test 1

In [22]:
num_qubit_states = 2

g_state = qt.basis(num_qubit_states, 0)
e_state = qt.basis(num_qubit_states, 1)

In [23]:
#same thing as a kroneker product between individual states
state1 = qt.tensor(g_state, g_state)
state2 = qt.tensor(g_state, e_state)
state3 = qt.tensor(e_state, g_state)
state4 = qt.tensor(e_state, e_state)

In [24]:
psi_i = np.array(state4)
psi_i

array([[0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [1.+0.j]])

In [25]:
concurrence(psi_i)

0.0

In [47]:
U = can_decomp(0.5, 0, 0)
U #CNOT locally equivalent gate

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        -0.70710678j]
 [0.        +0.j         0.70710678+0.j         0.        -0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.70710678j 0.70710678+0.j
  0.        +0.j        ]
 [0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.70710678+0.j        ]]

In [62]:
CNOT = qt.Qobj(np.array([[1 , 0 , 0 , 0 ],
                   [0 , 1 , 0 , 0 ],
                   [0 , 0 , 0 , 1 ],
                   [0 , 0 , 1 , 0]]))

In [48]:
psi_f1 = qt.Qobj(U*psi_i)
psi_f1

Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[0.        -0.70710678j]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.70710678+0.j        ]]

In [65]:
CNOT*psi_i

array([[0.+0.j],
       [0.+0.j],
       [1.+0.j],
       [0.+0.j]])

In [66]:
concurrence(psi_f1)

0.9999999999999998

In [67]:
concurrence(CNOT*psi_i)

0.0

In [69]:
x1 = randSU2()
y1 = randSU2()

x2 = randSU2()
y2 = randSU2()

In [70]:
#spans SU(2) kron SU(2)
xy1 = qt.Qobj(np.kron(x1, y1))
xy2 = qt.Qobj(np.kron(x2, y2))

In [52]:
U_prime = xy1*U*xy2 #multiply a canonical gate directly on both sides with two sets of two single qubit gates
U_prime

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.69801438-0.38963073j  0.10894219-0.31162805j  0.06082144-0.06831944j
   0.24262832+0.42982332j]
 [ 0.3271533 +0.05870335j -0.74946767+0.27734003j -0.4066918 +0.27698997j
  -0.07464308+0.05667539j]
 [-0.18258164-0.28250716j -0.31777973-0.19455866j  0.48202599+0.08309498j
  -0.65067902+0.2921979j ]
 [ 0.15613623-0.33610055j  0.13879958+0.30707967j -0.33660789-0.63004194j
  -0.46063088-0.16326702j]]

In [53]:
psi_f2 = qt.Qobj(U_prime*psi_i)
concurrence(psi_f2)

0.38691978679021266

In [54]:
psi_f2 #clearly coefficients are not real...

Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[ 0.24262832+0.42982332j]
 [-0.07464308+0.05667539j]
 [-0.65067902+0.2921979j ]
 [-0.46063088-0.16326702j]]

If I haven't made a serious mistake somewhere, U_prime and U should be locally equivalent as they differ solely by single qubit gates... why is the concurrence of the final wave function different??? Either I've made a serious error somewhere in my code or I misread statements made in papers